In [1]:
# header
import pandas as pd
import AddIPAFlex
import MatchHeadwords
from GenerateLexDir import literal_eval_col
from IPython.display import display

# load in df's
flexicon = pd.read_csv('flexiconFLAT.csv', index_col='entry_id', keep_default_na=False)
senses = pd.read_csv('SensesMERGE.csv', index_col='sense_id', keep_default_na=False)

# take things literally
literal_eval_col(flexicon, 'sense')
literal_eval_col(flexicon, 'note')
literal_eval_col(flexicon, 'other_sources')
literal_eval_col(flexicon, 'these_vars')
literal_eval_col(flexicon, 'variant_of')

literal_eval_col(senses, 'gloss')
literal_eval_col(senses, 'def')

In [11]:
# match function expects 'ipa' key in row
# create ipa col in flexicon
bibs = [x if x else 'eppsob' for x in flexicon['bibliography']]
ipa = [AddIPAFlex.to_ipa(hdwd, bib=bib) for hdwd, bib in zip(flexicon['headword'],\
                                                         bibs)]
assert all(x for x in ipa)
flexicon.loc[:, 'ipa'] = ipa

In [12]:
hdwrds = [not x for x in flexicon['variant_of']]
hdwrds = flexicon[hdwrds]
# O(thicc)
match = MatchHeadwords.match_dfs(hdwrds, hdwrds)
not_empty = [bool(x) for x in match['matches']]
match = match[not_empty]
match.to_csv('merge_matches2_29.csv')
match

x000002299A0A81F8> 0.2224256992340088
<function get_matches at 0x000002299A0A81F8> 0.35877370834350586
<function get_matches at 0x000002299A0A81F8> 0.2394096851348877
<function get_matches at 0x000002299A0A81F8> 0.27528858184814453
<function get_matches at 0x000002299A0A81F8> 0.32018613815307617
<function get_matches at 0x000002299A0A81F8> 0.3752458095550537
<function get_matches at 0x000002299A0A81F8> 0.3340611457824707
<function get_matches at 0x000002299A0A81F8> 0.3749508857727051
<function get_matches at 0x000002299A0A81F8> 0.25632429122924805
<function get_matches at 0x000002299A0A81F8> 0.303189754486084
<function get_matches at 0x000002299A0A81F8> 0.21746301651000977
<function get_matches at 0x000002299A0A81F8> 0.3580801486968994
<function get_matches at 0x000002299A0A81F8> 0.4208967685699463
<function get_matches at 0x000002299A0A81F8> 0.2523512840270996
<function get_matches at 0x000002299A0A81F8> 0.3181180953979492
<function get_matches at 0x000002299A0A81F8> 0.264294385910034

,matches
entry_id,
atsóm_02a79639-e72e-482a-98fe-e2a3e76d3f81,{'acõb_f71894c0-8fab-4c50-85f140d22a323e73': ...
naëng_031a7339-0644-4618-af27-88f71d6565db,{'nahə:h_b54a9153-964b-464c-a3e572767ea0d316':...
asooh_045c762b-0b2e-4233-a013-7e921270d050,{'aʃu:_844f59f2-271d-44ac-98c625f13b1b75d0': (...
baad_068ae71c-c922-438c-9569-dad5b2122f74,{'pëëd_575e234e-0e14-44a8-a947-b90d4f165f7b': ...
madyyk_09e54f3c-410b-415c-8850-75ea270e4552,{'badɨ:k_ec4b457c-f57d-40da-9afc1a28497674a1':...
...,...
waw&apos;oos_713c2e49-683d-4b4c-b21a-cb6fbd07c005,{'wawo̰:ʃ_713c2e49-683d-4b4c-b21acb6fbd07c006'...
wëëm_990d5cb5-98be-420d-9662-a39d1662fc24,{'wëm_429bd290-09e2-451d-876e-1e9f8f1d97e2': (...
wooh_26bc7f23-fb0b-4d30-93e4-da9a7933066d,{'mooh1_f3b52f1a-d90c-47a3-a82e-0f14c1209ae3':...


In [4]:
# let's reshape the file a bit
# instead of matching each id on the left to a dict of various correspondences,
# have each correspondence be a separate row
match = pd.read_csv('merge_matches2_29.csv', keep_default_na=False)
literal_eval_col(match, 'matches')
new_matches = pd.DataFrame(columns=['form', 'def', 'match_form', 'match_def', 'status', 'entry_id', 'match_id' ])
new_matches

,form,def,match_form,match_def,status,entry_id,match_id


In [5]:
for index, row in match.iterrows():
    data = dict(row)
    form = flexicon.at[row['entry_id'], 'headword']
    data['form'] = form
    data['status'] = ''
    this_match = data.pop('matches')
    for key, value in this_match.items():
        new_matches.loc[ len(new_matches) ] = {'match_id': key, 'match_form': value[0], 'match_def': value[1], **data }
new_matches.shape

(1400, 7)

In [7]:
new_matches.head()

,form,def,match_form,match_def,status,entry_id,match_id
0,atsóm,swim;nadar,acõb,ura/berno,,atsóm_02a79639-e72e-482a-98fe-e2a3e76d3f81,acõb_f71894c0-8fab-4c50-85f140d22a323e73
1,naëng,water;água,nahə:h,doença,,naëng_031a7339-0644-4618-af27-88f71d6565db,nahə:h_b54a9153-964b-464c-a3e572767ea0d316
2,asooh,sit;sentar,aʃu:,costurar,,asooh_045c762b-0b2e-4233-a013-7e921270d050,aʃu:_844f59f2-271d-44ac-98c625f13b1b75d0
3,baad,well;bem,pëëd,?;cunuri,,baad_068ae71c-c922-438c-9569-dad5b2122f74,pëëd_575e234e-0e14-44a8-a947-b90d4f165f7b
4,baad,well;bem,pa:n,igual,,baad_068ae71c-c922-438c-9569-dad5b2122f74,pa:n_e4920278-36b2-4b6c-b2669c72f3055ccf


In [9]:
# let's get rid of duplicates
for index, row in new_matches.copy().iterrows():
    id = row['entry_id']
    match_id = row['match_id']
    for this_index, this_row in new_matches.copy().iterrows():
        this_id = this_row['entry_id']
        this_match_id = this_row['match_id']
        if index != this_index and id == this_match_id and match_id == this_id and index in new_matches.index:
            new_matches = new_matches.drop(this_index)

In [10]:
new_matches.shape

(700, 7)

In [41]:
new_matches.to_csv('merge_matches3_1.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'merge_matches3_1.csv'

In [2]:
matches = pd.read_csv('merge_matches3_1.csv', index_col='entry_id', keep_default_na=False)

In [3]:
keep = matches[[x == 'merge' for x in matches['status']]]
keep

,form,match_form,def,match_def,status,match_id
entry_id,,,,,,
jawëk_11b99d8c-9b68-4e6b-869d-e3530425e72b,jawëk,jawäk,japurá;japurá,japurá;japurá,merge,jawäk_11b99d8c-9b68-4e6b-869d-e3530425e72b
janati1_1bd7a005-8171-46fd-a65d-69898076947f,janati,janati,maize;milho,maize;milho,merge,janati2_44596d86-91ab-40aa-bd2d-c32eead8c0bf
janati1_1bd7a005-8171-46fd-a65d-69898076947f,janati,janati,maize;milho,maize;milho,merge,janati_1bd7a005-8171-46fd-a65d-69898076947f
gawajah_1c43c5f6-18af-437e-90c9-8506069bb3d1,gawajah,gawajaah,,hunt;caçar,merge,gawajaah3_473237ec-7ec5-4fc0-88e7-d5033497445e
gawajah_1c43c5f6-18af-437e-90c9-8506069bb3d1,gawajah,gawaja:h,,correr.atrás.de.caça,merge,gawaja:h_a2ce689e-bdf8-4a72-86b52372cff080e9
kanés g’aad_2ee0c600-0fc1-45ee-803e-4332e97219c8,kanés g’aad,kanés gʹaad,roof.thatch.palm;caraná,carana.leaf;folha;caraná,merge,kanés gʹaad_338788c5-1d3d-44d0-a465-c5ff6230dc7d
w’oor_3430ca49-d3b7-4b34-bff7-96d27c9662e2,w’oor,w'oor,owl;coruja,owl;coruja,merge,w&apos;oor_3430ca49-d3b7-4b34-bff7-96d27c9662e2
k’ããts_3a94f9ae-6b60-4f40-8241-3d12b67a67be,k’ããts,ga̰c,soil;terra,areia,merge,ga̰c_07f4b7eb-2a95-4855-9a4d8837b5a4006f
t’õõm_3cdcf804-ed94-4d32-8388-d8ed3cf4b239,t’õõm,t'õõm,disposable.basket;cesto.descartável,woven.traditional.backpack;panacu,merge,t&apos;õõm_6945fd07-f2f1-4af1-a00a-c25ab150e2a3


In [46]:
# most efficient way to deal w/ these bois is to write a csv where all rows to be merged are consecutive
groups = [] # list to store groups of row ids


for id, m_id in zip(keep.index, keep['match_id']):
    found = False # each id should only belong to one group
    for g in groups:
        if id in g:
            assert not found, groups
            g.append(m_id)
            found = True
        elif m_id in g:
            assert not found, groups
            g.append(id)
            found = True
    if not found:
        groups.append( [id, m_id] )

len(groups)

51

In [47]:
out = pd.DataFrame( columns = flexicon.columns )
out.insert(0, 'entry_id', [])
for g in groups:
    for id in g:
        row = dict(flexicon.loc[id])
        row['entry_id'] = id
        out.loc[len(out)] = row
    out.loc[len(out)] = {colname: '' for colname in out.columns}

for i, sense in enumerate(out['sense']):
    if not sense:
        continue
    sense = sense[:]
    for j, id in enumerate(sense):
        sense[j] = dict( senses.loc[id] )
    out.at[i, 'sense'] = sense

In [48]:
out

,entry_id,bibliography,date,date_modified,headword,morph_type,note,other_sources,pronunciation,sense,these_vars,variant_of
0,jawëk_11b99d8c-9b68-4e6b-869d-e3530425e72b,Fieldnotes Epps/Obert; 2018,2018-09-04T00:12:46Z,,jawëk,root,{},None,jaˈwǝk˺,"[{'pos': 'Noun', 'gloss': {'en': 'japurá', 'pt...",None,{}
1,jawäk_11b99d8c-9b68-4e6b-869d-e3530425e72b,Epps/Obert Fieldnotes 2018,2018-09-04T00:12:46Z,2020-01-08T15:52:38Z,jawäk,root,{'bibliography': 'Fieldnotes Epps/Obert; 2018'},{},jaˈwʌk˺,"[{'pos': 'Noun', 'gloss': {'en': 'japurá', 'pt...",{},None
2,,,,,,,,,,,,
3,janati1_1bd7a005-8171-46fd-a65d-69898076947f,Fieldnotes Epps/Obert; 2018,2018-09-03T23:46:33Z,,janati,root,{},None,janaˈtiʔ,"[{'pos': 'Noun', 'gloss': {'en': 'maize', 'pt'...",None,{}
4,janati2_44596d86-91ab-40aa-bd2d-c32eead8c0bf,Fieldnotes Epps/Obert; 2018,2019-01-14T03:46:55Z,,janati,stem,None,None,jadnãtiʔ,"[{'pos': 'Noun', 'gloss': {'en': 'maize', 'pt'...",None,{}
...,...,...,...,...,...,...,...,...,...,...,...,...
157,ti1_6e529536-55b7-49da-b65f-093cf90a0686,Epps/Obert Fieldnotes 2019,2018-03-08T07:17:53Z,2020-01-26T04:38:27Z,ti,stem,"{'bibliography': 'Epps/Obert; fieldnotes 2019,...",{},ti,"[{'pos': 'conj', 'gloss': {'en': 'and.then', '...",{},None
158,,,,,,,,,,,,
159,pɛh_ee911202-c9b9-436e-8f4e23aff9b0cbb7,SIL Dict 2011,2018-03-08T07:17:53Z,,pɛh,stem,{'Note': 'Predicted phonemic form from source ...,"{'Weir': 'péh', 'Sil Dict 2011': 'péh'}",pɛh,"[{'pos': 'n', 'gloss': {'pt': 'tipo.de'}, 'def...",{'péh2_185742c8-a9c5-4745-b66a-a8f4f48aaf5c': ...,{}
160,péh_ee911202-c9b9-436e-8f4e-23aff9b0cbb6,SIL Dict 2011,2018-03-08T07:17:53Z,2019-12-04T17:33:35Z,péh,stem,"{'bibliography': 'SIL 2011', 'Note': ''non-ref...",{},pɛh,"[{'pos': 'n', 'gloss': {'pt': 'tipo.de'}, 'def...",{},None


In [49]:
out.to_csv('matches-organized.csv', index=False)

In [4]:
# cool beans, lets make the changes 
merges = pd.read_csv('matches-organized.csv', index_col='entry_id', keep_default_na=False)

In [5]:
# let's start by replacing the sense col w/ the original sense id
for index, sense in zip( merges.index, merges['sense'] ):
    if '{' in sense:
        merges.at[index, 'sense'] = flexicon.at[index, 'sense']

merges = merges[ [bool(x) for x in merges.index] ]

In [9]:
flexicon = backup

In [10]:
# now we can just drop everything from keep
backup = flexicon.copy()
print(flexicon.shape)

to_drop = set( list( keep.index ) )

for index in to_drop:
    assert index in flexicon.index, index
    flexicon = flexicon.drop(index)

print(flexicon.shape)

# and add the rows from merges

for index in merges.index:
    #print(merges.loc[index])
    flexicon.loc[index] = merges.loc[index]

print(flexicon.shape)

(1188, 11)
(1135, 11)
(1157, 11)


In [11]:
# save the result
flexicon.to_csv('flexicon_3_3.csv')